# **Presentación del caso: Análisis de Sentimientos con Twitter**

Este caso muestra un conjunto de **13,962 tweets** en la data de entrenamiento. Cuenta con una la columna *label* (etiqueta u objetivo) donde se registra **dos valores**:
1.   0: tweet positivo sin mención de temas políticos, racistas, etc
2.   1: tweet negativo que menciona temas políticos, racistas, etc
![Image of Yaktocat](https://s3.amazonaws.com/thinkific/file_uploads/118220/images/0c8/229/64d/1549269497113.jpg)

Para tratar la información contenida en los tweets, **data no estructurada**, procederemos a utilizar la librería **nltk** -Natural Language Tokenization- y los siguientes modelos de NLP -Natural Language Processing para procesar los textos:
1.   BOW: Bag of Words
2.   TF-IDF
3.   W2V: Word to Vec

Finalmente para realizar las predicciones complementamos a los modelos de NLP antes mencionados varios modelos de clasificación para predecir la columna label por tratarse de dos categorías (0 y 1).


Los detalles de la competencia **gratuita** pueden encontrarse en la siguiente [web](https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/)

# **Comprensión de los datos**

In [ ]:
"CASO N°1"
#Importamos las librerías necesarias 
import pandas as pd
import re    # regular expression nos permite editar la visualización de los caracteres según nuestras prefencias
import nltk  # natural language tokenization es una librería para manipular las palabras que encontraremos en los tweets
import string # exclusivamente para palabras 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_colwidth", 200) #Definimos un máximo de ancho de columna por la naturaleza del largo de los textos

In [ ]:
"CELDA N°2"
#Leemos los datos de entrenamiento -train- y los datos de testing -test
train=pd.read_csv('https://raw.githubusercontent.com/javalpe/datasets/master/train_twitter_analysis.csv', index_col="id")
summit=pd.read_csv('https://raw.githubusercontent.com/javalpe/datasets/master/test_twitter_analysis.csv', index_col="id")

In [ ]:
"CELDA N°3"
#Visualizamos el tamaño de cada dataset con el comando shape
train.shape, summit.shape

In [ ]:
"CELDA N°4"
#Visualizamos el contenido de los tweets con el label = 0 que serían considerados positivos
train[train['label'] == 0].head(10)

In [ ]:
"CELDA N°5"
#Visualizamos el contenido de los tweets con el label = 1 que serían considerados negativos
train[train['label'] == 1].head(10)

In [ ]:
"CELDA N°6"
#Podemos distinguir la cantidad de tweets positivos y negativos con el método value_counts()
train["label"].value_counts()

Como la cantidad de tweets positivos es **15 veces mayor** de los tweets negativos podemos concluir que tenemos **data desbalanceada**. 

In [ ]:
"CELDA N°7"
#Visualizamos el largo de los tweets -número de caracteres- en train y test con el método str.len()
length_train = train['tweet'].str.len() 
length_summit = summit['tweet'].str.len() 
plt.hist(length_train, bins=20, label="train_tweets") #el método hist de la librería matplotlib (plt) permite dibujar histogramas
plt.hist(length_summit, bins=20, label="summit_tweets") #el parámetro label es para identificar estos datos para una leyenda del gráfico
plt.legend() #el comando legend permite añadir una leyenda a nuestra gráfica
plt.show()

Adicionalmente se visualiza que la cantidad de caracteres de los tweets positivos es mucho mayor que los tweets negativos. Por ello podemos concluir que el modelo final será **más preciso para predecir tweets positivos que negativos**

# **Preprocesamiento de los datos**

## **Extracción de los caracteres especiales**

In [ ]:
"CELDA N°8"
#Inicialmente juntamos train y test para realizar este tratamiento para ambos dataset. Luego volveremos a dividirlos
combi = train.append(summit, ignore_index=True) 
combi.shape

In [ ]:
"CELDA N°9"
#Definimos una función para eliminar de los tweets alguna palabra, simbolo o caracter especial (que irá en el parámetro pattern) que creamos conveniente
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [ ]:
"CELDA N°10"
#Quitamos todas aquellas menciones dentro de los tweets a otros usuarios (que comienzan con @)
combi['tweet_limpio'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")
combi.head()

In [ ]:
"CELDA N°11"
#Reemplazamos todos aquellos carácteres especiales como signos de puntuación por un espacio en blanco
combi['tweet_limpio'] = combi['tweet_limpio'].str.replace("[^a-zA-Z#]", " ") #el simbolo ^ signfica "todo menos" en este caso palabras y números 
combi.head(10)

In [ ]:
"CELDA N°12"
#Removemos todas aquellas palabras con un largo de 3 caracteres o menos que podrían incluir conectores, advervios, artículos (por ejemplo a, and, the)
combi['tweet_limpio'] = combi['tweet_limpio'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
combi.head(10)

## **Tokenización y Steamming**

Para poder evaluar cada tweet será necesario dividirlo en palabras, este proceso se denomina *tokenization*.

También realizaremos el proceso denominado *steamming* para solo utilizar las raíces de las palabras en el análisis (por ejemplo "read" en vez de "reading")

In [ ]:
"CELDA N°13"
#Dividimos los tweets en palabras y guardamos el resultado en una nueva columna denominada tweet_limpio
tokenized_tweet = combi['tweet_limpio'].apply(lambda x: x.split()) #el comando split signfica dividir
tokenized_tweet.head()

In [ ]:
"CELDA N°14"
#Ahora procederemos a extraer solo la raíz de las palabras con el método stem.porter y actualizamos la columna tweet_limpio
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
stemmed_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [ ]:
"CELDA N°15"
#Finalmente juntamos cada raíz de palabra para almacenarla nuevamente en la columna tweet_limpio
for i in range(len(stemmed_tweet)):
    stemmed_tweet[i] = ' '.join(stemmed_tweet[i])    
combi['tweet_limpio'] = stemmed_tweet

## **Aplicaciones de visualización**

### 1. *WORDCLOUD*

In [ ]:
"CELDA N°16"
#Importamos la librería WordCloud para visualizar las palabras más usadas (que aparecen en mayor tamaño)
from wordcloud import WordCloud

In [ ]:
"CELDA N°17"
#Mostramos las palabras más usadas
all_words = ' '.join([text for text in combi['tweet_limpio']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words) 
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
"CELDA N°18"
#Mostramos las palabras más usadas en los tweets positivos
normal_words =' '.join([text for text in combi['tweet_limpio'][combi['label'] == 0]]) 
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(normal_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
"CELDA N°19"
#Mostramos las palabras más usadas en los tweets negativos
negative_words =' '.join([text for text in combi['tweet_limpio'][combi['label'] == 1]]) 
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(negative_words)
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

### 2. *HASHTAG MÁS USADOS*

In [ ]:
"CELDA N°20"
# Definimos una función que nos permita extraer los hashtags (#) de los tweets
def hashtag_extract(x):
    hashtags = []
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)
    return hashtags

In [ ]:
"CELDA N°21"
#Extraemos los hashtags (#) de los tweets que no son racistas ni sexistas (label=0) y guardamos el resultado en la variable HT_positivos
HT_positivos = hashtag_extract(combi['tweet_limpio'][combi['label'] == 0])

In [ ]:
"CELDA N°22"
#Extraemos los hashtags (#) de los tweets que sí son racistas ni sexistas (label=1) y guardamos el resultado en la variable HT_negativos 
HT_negativos = hashtag_extract(combi['tweet_limpio'][combi['label'] == 1])

In [ ]:
"CELDA N°23"
#Para poder realizar un conteo y graficarlo en barras procedemos a unir todas las palabras de cada variable
HT_positivos = sum(HT_positivos,[])
HT_negativos = sum(HT_negativos,[])

In [ ]:
"CELDA N°24"
#Con ayuda del método FreqDist realizamos el conteo y graficamos las frecuencias de aparición de cada hashtag en los tweets de la variable HT_regular (label=0)
a = nltk.FreqDist(HT_positivos)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})
d = d.nlargest(columns="Count", n = 10) #Para efectos de visualización solo nos quedaremos con el top10
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count") #Para
ax.set(ylabel = 'Count')
plt.show()

In [ ]:
"CELDA N°25"
#Con ayuda del método FreqDist realizamos el conteo y graficamos las frecuencias de aparición de cada hashtag en los tweets de la variable HT_negative (label=1)
b = nltk.FreqDist(HT_negativos)
e = pd.DataFrame({'Hashtag': list(b.keys()), 'Count': list(b.values())})
e = e.nlargest(columns="Count", n = 10)   #Para efectos de visualización solo nos quedaremos con el top10
plt.figure(figsize=(16,5))
ax = sns.barplot(data=e, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

## **Bag of Words (BOW)**

El primer método para preparar la data y poder ejecutar modelos de clasificación se denomina **Bag of Words** donde se obtiene un dataset compuesto por D documentos (que representan filas o registros) y N corpus (que representan las variables predictoras)

Para nuestro ejemplo el objetivo es obtener como variables predictoras (columnas o *features*) las palabras y las filas o registros serán cada tweet.

In [ ]:
"CELDA N°26"
#Importamos la librería CountVectorizer de sklearn
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(analyzer='word',stop_words='english') #en nuestro caso estamos procesando tweets en inglés
bow = bow_vectorizer.fit_transform(combi['tweet_limpio']) #guardamos el resultado en una variable denominada bow

## **TF - IDF**

El segundo método para preparar la data y poder ejecutar modelos de clasificación se denomina **TF - IDF** donde se obtiene un dataset conformado por los mismos elementos. Sin embargo vamos a añadir dos elementos de evaluación (como pesos para cada variable):


1.   TF = # veces que aparece una palabra en un solo documento / # palabras totales de ese mismo documento
2.   IDF = log (N/n) n es el # veces que aparece una palabra en cada documento y N es el # total de documentos


Para nuestro ejemplo recordemos que un documento es un tweet y una palabra es cada una de las palabras que hemos tokenizado y steammeado.

In [ ]:
"CELDA N°27"
#Importamos la librería TfidVectorizer de sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words='english') #en nuestro caso estamos procesando tweets en inglés
tfidf = tfidf_vectorizer.fit_transform(combi['tweet_limpio']) #guardamos el resultado en una variable denominada tfidf

## **Word2Vec (w2v)**

El tercer método para preparar la data y poder ejecutar modelos de clasificación se denomina **Word2Vec** donde cada palabra se transforma en un vector para relacionar las palabras entre sí antes de generar el dataset. Esto nos trae dos ventajas:


1.   Se reduce significativamente la cantidad de columnas generadas
2.   Se puede distinguir dos significados distintos para una misma palabra (por ejemplo *apple* puede significar fruta y también signficar empresa tecnológica)

In [ ]:
"CELDA N°28"
#Dividimos nuevamente la columna tweet_limpio palabra por palabra para poder realizar el análisis y guardamos el resultado en la variable tokenized_tweet
tokenized_tweet = combi['tweet_limpio'].apply(lambda x: x.split())

In [ ]:
"CELDA N°29"
#Importamos las librerías que necesitamos para ejecutar el análisis Word2vec
import gensim
from gensim.models.doc2vec import LabeledSentence

In [ ]:
"CELDA N°30"
#Generamos un modelo word2vec y guardamos el modelo en la variable model_w2v
model_w2v = gensim.models.Word2Vec(tokenized_tweet,
                                   window=5,
                                   size= 300,
                                   min_count=2,
                                   alpha=0.03,
                                   min_alpha=0.0007,
                                   negative=20)

In [ ]:
"CELDA N°31"
#Entrenamos el modelo con los tweets tokenizados
model_w2v.train(tokenized_tweet, total_examples= len(combi['tweet_limpio']), epochs=15)

In [ ]:
"CELDA N°32"
#Definimos una función para generar un vector para cada uno de los tweet
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: continue
        if count != 0: vec/= count
    return vec

In [ ]:
"CELDA N°33"
#Aplicamos la función sobre cada tweet y generamos un dataframe que guardaremos como la variable wordvec_df
wordvec_arrays = np.zeros((len(tokenized_tweet), 300)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 300)
    wordvec_df = pd.DataFrame(wordvec_arrays)

In [ ]:
"CELDA N°34"
#Comprobamos el tamaño de cada variable creada
bow.shape, tfidf.shape, wordvec_df.shape

# **Modelamiento**

In [ ]:
"CELDA N°35"
#Dividimos la variable bow en train y summit originales
train_bow = bow[:31962,:] #Esta parte de la data sí contiene el label
summit_bow = bow[31962:,:]

In [ ]:
"CELDA N°36"
#Dividimos la variable tfidf en train y summit originales
train_tfidf = tfidf[:31962,:] #Esta parte de la data sí contiene el label
summit_tfidf = tfidf[31962:,:]

In [ ]:
"CELDA N°37"
#Dividimos la variable wordvec_df en train y summit originales
train_w2v = wordvec_df.iloc[:31962,:] #Esta parte de la data sí contiene el label
summit_w2v = wordvec_df.iloc[31962:,:]

In [ ]:
"CELDA N°38"
#Utilizamos el método train_test_split para dividir train y test
from sklearn.model_selection import train_test_split
Xtrain_bow, Xtest_bow, ytrain_bow, ytest_bow = train_test_split(train_bow, train['label'], test_size=0.15)
Xtrain_tfidf, Xtest_tfidf, ytrain_tfidf, ytest_tfidf = train_test_split(train_tfidf, train['label'], test_size=0.15)
Xtrain_w2v, Xtest_w2v, ytrain_w2v, ytest_w2v = train_test_split(train_w2v, train['label'], test_size=0.15)

In [ ]:
"CELDA N°39"
#Importamos las librerías útiles para todos los modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## **Regresión Logística**

In [ ]:
"CELDA N°40"
#Importamos las librerías de Regresión Logística y guardamos el modelo de Regresión Logística en la variable lreg
from sklearn.linear_model import LogisticRegression
lreg = LogisticRegression()

### 1. *Regresión Logística a través de BOW*

In [ ]:
"CELDA N°41"
#Entrenamos el modelo de Regresión Logística con Xtrain_bow, ytrain_bow y generamos una lista de accuracy score a partir de Xtest_bow, ytest_bow
lreg.fit(Xtrain_bow, ytrain_bow)
proba_rl_bow = lreg.predict_proba(Xtest_bow)[:,1]
list_accuracy_bow=[]
for punto_de_corte in range(0,100):
    pred_rl_bow = [1 if x >= punto_de_corte/100 else 0 for x in proba_rl_bow]
    list_accuracy_bow.append(accuracy_score(ytest_bow, pred_rl_bow))

In [ ]:
"CELDA N°42"
#Dibujamos los puntos de corte con sus respectivos score
xs = [x/100 for x in range(0,100)]
ys = list_accuracy_bow

plt.figure(figsize=(8,6))
plt.grid(True)
plt.xlabel('Punto de corte')
plt.ylabel('list_accuracy_bow')
plt.plot(xs, ys)

In [ ]:
"CELDA N°43"
#Actualizamos las predicciones de BOW con el punto de corte óptimo
pred_rl_bow = proba_rl_bow >= 0.19
pred_rl_bow = pred_rl_bow.astype(np.int)

### 2. *Regresión Logística a través de TF-IDF*

In [ ]:
"CELDA N°44"
#Entrenamos el modelo de Regresión Logística con Xtrain_tfidf, ytrain_tfidf y generamos una lista de accuracy score a partir de Xtest_tfidf, ytest_tfidf
lreg.fit(Xtrain_tfidf, ytrain_tfidf)
proba_rl_tfidf = lreg.predict_proba(Xtest_tfidf)[:,1]
list_accuracy_tfidf=[]
for punto_de_corte in range(0,100):
    pred_rl_tfidf = [1 if x >= punto_de_corte/100 else 0 for x in proba_rl_tfidf]
    list_accuracy_tfidf.append(accuracy_score(ytest_tfidf, pred_rl_tfidf))

In [ ]:
"CELDA N°45"
#Dibujamos los puntos de corte con sus respectivos score
xs = [x/100 for x in range(0,100)]
ys = list_accuracy_tfidf

plt.figure(figsize=(8,6))
plt.grid(True)
plt.xlabel('Punto de corte')
plt.ylabel('list_accuracy_tfidf')
plt.plot(xs, ys)

In [ ]:
"CELDA N°46"
#Actualizamos las predicciones de TF-IDF con el punto de corte óptimo
pred_rl_tfidf = proba_rl_tfidf >= 0.17
pred_rl_tfidf = pred_rl_tfidf.astype(np.int)

### 3. *Regresión Logística a través de W2V*

In [ ]:
"CELDA N°47"
#Entrenamos el modelo de Regresión Logística con Xtrain_w2v, ytrain_w2v y generamos una lista de accuracy score a partir de Xtest_w2v, ytest_w2v
lreg.fit(Xtrain_w2v, ytrain_w2v)
proba_rl_w2v = lreg.predict_proba(Xtest_w2v)[:,1]
list_accuracy_w2v=[]
for punto_de_corte in range(0,100):
    pred_rl_w2v = [1 if x >= punto_de_corte/100 else 0 for x in proba_rl_w2v]
    list_accuracy_w2v.append(accuracy_score(ytest_w2v, pred_rl_w2v))

In [ ]:
"CELDA N°48"
#Dibujamos los puntos de corte con sus respectivos score
xs = [x/100 for x in range(0,100)]
ys = list_accuracy_w2v

plt.figure(figsize=(8,6))
plt.grid(True)
plt.xlabel('Punto de corte')
plt.ylabel('list_accuracy_w2v')
plt.plot(xs, ys)

In [ ]:
"CELDA N°49"
#Actualizamos las predicciones de W2V con el punto de corte óptimo
pred_rl_w2v = proba_rl_w2v >= 0.18
pred_rl_w2v = pred_rl_w2v.astype(np.int)

## **RandomForest**

In [ ]:
"CELDA N°50"
#Importamos la librería de RandomForestClassifier y guardamos el modelo de Random Forest en la variable rf
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(max_depth=3)

### 1. *Random Forest a través de BOW*

In [ ]:
"CELDA N°51"
#Aplicamos el modelo de Random Forest en Xtrain_bow, ytrain_bow y generamos las predicciones a partir de Xtest_bow
rf_bow = rf.fit(Xtrain_bow, ytrain_bow)
pred_rf_bow = rf_bow.predict(Xtest_bow)

### 2. *Random Forest a través de TF-IDF*

In [ ]:
"CELDA N°52"
#Aplicamos el modelo de Random Forest en Xtrain_tfidf, ytrain_tfidf y generamos las predicciones a partir de Xtest_tfidf
rf_tfidf = rf.fit(Xtrain_tfidf, ytrain_tfidf)
pred_rf_tfidf = rf_tfidf.predict(Xtest_tfidf)

### 3. *Random Forest a través de W2V*

In [ ]:
"CELDA N°53"
#Aplicamos el modelo de Random Forest en Xtrain_w2v, ytrain_w2v y generamos las predicciones a partir de Xtest_w2v
rf_w2v = rf.fit(Xtrain_w2v, ytrain_w2v)
pred_rf_w2v = rf_w2v.predict(Xtest_w2v)

## **XGBoost**

In [ ]:
"CELDA N°54"
#Importamos la librería de XGBClassifier y guardamos el modelo de XGBoost en la variable xgb
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate = 0.1,
                      early_stopping_rounds = 10,
                      min_child_weight=2,
                      gamma=0.1,
                      subsample=0.85,
                      colsample_bytree=0.8,
                      objective= 'binary:logistic') 

### 1. *XGBoost a través de BOW*

In [ ]:
"CELDA N°55"
#Aplicamos el modelo de XGBClassifier para los datos procesados según BOW 
xgb_bow = xgb.fit(Xtrain_bow, ytrain_bow) 
pred_xgb_bow = xgb_bow.predict(Xtest_bow)

### 2. *XGBoost a través de TF-IDF*

In [ ]:
"CELDA N°56"
#Aplicamos el modelo de XGBClassifier para los datos procesados según TF-IDF 
xgb_tfidf = xgb.fit(Xtrain_tfidf, ytrain_tfidf) 
pred_xgb_tfidf = xgb_tfidf.predict(Xtest_tfidf)

### 3. *XGBoost a través de W2V*

In [ ]:
"CELDA N°57"
#Aplicamos el modelo de XGBClassifier para los datos procesados según W2V 
xgb_w2v = xgb.fit(Xtrain_w2v, ytrain_w2v) 
pred_xgb_w2v = xgb_w2v.predict(Xtest_w2v)

# **Evaluación de modelos**

In [ ]:
"CELDA N°58"
#Definimos una lista de métricas y creamos una función para calcular todas las métricas
lista_metricas=[accuracy_score,precision_score,recall_score,f1_score,roc_auc_score]
def calcula_scores_modelo(score_modelo_nlp,real,prediccion):
  for metrica in lista_metricas: score_modelo_nlp.append(metrica(real, prediccion))

In [ ]:
"CELDA N°59"
#Calculamos todas las métricas para el modelo de Regresión Logística con cada procesamiento de NLP: BOW, TF-IDF, W2V
score_rl_bow=[]
calcula_scores_modelo(score_rl_bow,ytest_bow,pred_rl_bow)

score_rl_tfidf=[]
calcula_scores_modelo(score_rl_tfidf,ytest_tfidf,pred_rl_tfidf)

score_rl_w2v=[]
calcula_scores_modelo(score_rl_tfidf,ytest_w2v,pred_rl_w2v)

In [ ]:
"CELDA N°60"
#Calculamos todas las métricas para el modelo de Random Forest con cada procesamiento de NLP: BOW, TF-IDF, W2V
score_rf_bow=[]
calcula_scores_modelo(score_rf_bow,ytest_bow,pred_rf_bow)

score_rf_tfidf=[]
calcula_scores_modelo(score_rf_tfidf,ytest_tfidf,pred_rf_tfidf)

score_rf_w2v=[]
calcula_scores_modelo(score_rf_w2v,ytest_w2v,pred_rf_w2v)

In [ ]:
"CELDA N°61"
#Calculamos todas las métricas para el modelo de XGBoost con cada procesamiento de NLP: BOW, TF-IDF, W2V
score_xgb_bow=[]
calcula_scores_modelo(score_xgb_bow,ytest_bow,pred_xgb_bow)

score_xgb_tfidf=[]
calcula_scores_modelo(score_xgb_tfidf,ytest_tfidf,pred_xgb_tfidf)

score_xgb_w2v=[]
calcula_scores_modelo(score_xgb_w2v,ytest_w2v,pred_xgb_w2v)

In [ ]:
"CELDA N°62"
#Generamos una tabla donde se consolide todos los indicadores
tabla_score = {'RegLogistica_BOW': score_rl_bow,'RegLogistica_TF_IDF': score_rl_tfidf, 'RegLogistica_W2V': score_rl_w2v,
               'RandomForest_BOW': score_rf_bow,'RandomForest_TF_IDF': score_rf_tfidf, 'RandomForest_W2V': score_rf_w2v,
               'XGBoost_BOW': score_xgb_bow,'XGBoost_TF_IDF': score_xgb_tfidf, 'XGBoost_W2V': score_xgb_w2v}
tabla_score = pd.DataFrame(tabla_score)  
tabla_score.index = ['accuracy_score','precision_score','recall_score','f1_score','roc_auc_score']
tabla_score

# **Summit de las predicciones**

In [ ]:
"CELDA N°63"
#Generamos las predicciones con el mejor modelo a partir de su respectivo summit
summit_pred = xgb_w2v.predict(summit_w2v)

In [ ]:
"CELDA N°64"
#Extraer los Id de los pasajeros del archivo summit original
origin = pd.read_csv('https://raw.githubusercontent.com/javalpe/datasets/master/test_twitter_analysis.csv')
summit_id=origin['id']

In [ ]:
"CELDA N°65"
#Comprobar que el tamaño de las predicciones y summmit_id coincidan para unir ambos dataset sin errores
summit_pred.shape, summit_id.shape

In [ ]:
"CELDA N°66"
#Crear un diccionario para guardar ambos dataset y crear un nuevo dataframe llamado respuestas para unir ambos dataset
summit_dict = {'id':summit_id,'label':summit_pred}
respuestas = pd.DataFrame(summit_dict)

In [ ]:
"CELDA N°67"
#Generar un archivo csv con tu nombre -no olvidar la extensión csv
respuestas.to_csv('nombre_apellido.csv',index=False)